# Learning to Rank на основе LETOR MQ2008
## Анализ данных, построение и оценка ранжирующей модели




In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("taoqin/letor4")

# print("Path to dataset files:", path)

Using Colab cache for faster access to the 'letor4' dataset.
Path to dataset files: /kaggle/input/letor4


In [ ]:
# !pip install optuna

In [ ]:
import os
import pandas as pd
import numpy as np
from lightgbm import LGBMModel, Dataset, train
from sklearn.metrics import ndcg_score
import optuna
from tqdm import tqdm

In [ ]:
fold_path = "/kaggle/input/letor4/MQ2008/Fold1"


print("Проверка наличия файлов:")
print("train.txt:", os.path.isfile(os.path.join(fold_path, "train.txt")))
print("vali.txt:", os.path.isfile(os.path.join(fold_path, "vali.txt")))
print("test.txt:", os.path.isfile(os.path.join(fold_path, "test.txt")))

Проверка наличия файлов:
train.txt: True
vali.txt: True
test.txt: True


In [ ]:
def load_letor_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split('#')[0].split()
            label = int(parts[0])
            qid = int(parts[1].split(':')[1])
            features = {f'feature_{i}': 0.0 for i in range(1, 47)}

            for elem in parts[2:]:
                if ':' in elem:
                    fid, value = elem.split(':')
                    fid = int(fid)
                    if 1 <= fid <= 46:
                        features[f'feature_{fid}'] = float(value)
                    else:
                      print(f"Недопуститмый признак. elem: {elem}, fid: {fid}" )

            row = {'label': label, 'qid': qid}
            row.update(features)
            data.append(row)

    df = pd.DataFrame(data)
    return df

In [ ]:
train_df = load_letor_data(os.path.join(fold_path, "train.txt"))
print(f"Тренировочный сет: {train_df.head(5)}")
print(f"Размер: {train_df.shape}")

vali_df = load_letor_data(os.path.join(fold_path, "vali.txt"))
print(f"Валидационный сет: {vali_df.head(5)}")
print(f"Размер: {vali_df.shape}")

test_df = load_letor_data(os.path.join(fold_path, "test.txt"))
print(f"Тестовый сет: {test_df.head(5)}")
print(f"Размер: {test_df.shape}")

Тренировочный сет:    label    qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0      0  10002   0.007477        0.0        1.0        0.0   0.007470   
1      0  10002   0.603738        0.0        1.0        0.0   0.603175   
2      0  10002   0.214953        0.0        0.0        0.0   0.213819   
3      0  10002   0.000000        0.0        1.0        0.0   0.000000   
4      0  10002   1.000000        1.0        0.0        0.0   1.000000   

   feature_6  feature_7  feature_8  ...  feature_37  feature_38  feature_39  \
0        0.0        0.0        0.0  ...    0.797056    0.697327    0.721953   
1        0.0        0.0        0.0  ...    0.000000    0.000000    0.117399   
2        0.0        0.0        0.0  ...    0.566409    0.760916    0.746370   
3        0.0        0.0        0.0  ...    0.320586    0.133604    0.000000   
4        0.0        0.0        0.0  ...    0.341228    0.292567    0.385569   

   feature_40  feature_41  feature_42  feature_43  feature_44

##EDA


In [ ]:
train_df['label'].value_counts()

,count
label,
0,7820
1,1223
2,587


In [ ]:
vali_df['label'].value_counts()

,count
label,
0,2140
1,400
2,167


In [ ]:
test_df['label'].value_counts()

,count
label,
0,2319
1,378
2,177


Наблюдается дисбаланс классов - класс 0 преобладает над остальными. Логично для специфики задачи - для конкретного запроса во всем интернете найдется больше нерелевантных документов, чем релевантных

In [ ]:
train_label_stats = train_df.groupby('qid')['label'].agg(['max']).value_counts()
train_label_stats

,count
max,
2,200
1,139
0,132


In [ ]:
vali_label_stats = vali_df.groupby('qid')['label'].agg(['size', 'max']).describe()
vali_label_stats

,size,max
count,157.000000,157.000000
mean,17.242038,1.197452
std,17.278414,0.796140
min,6.000000,0.000000
25%,8.000000,1.000000
50%,8.000000,1.000000
75%,16.000000,2.000000
max,118.000000,2.000000


In [ ]:
test_label_stats = test_df.groupby('qid')['label'].agg(['max']).value_counts()
test_label_stats

,count
max,
2,63
0,51
1,42


Можно увидеть, что для некотрых запросов отсутсвуют релевантные документы и в таком случае, задача ранжирования для них неактуальна, однако не стоит удалять эти запросы из наших данных так, как в релаьном мире пользователи также могут оставлять некорректные запросы и наша цель - сформировать хоть какую-то выдачу. Также если мы удалим такие запросы, то поднимем итоговые метрики, однако они будут неактуальны потому, что, как я и сказал раньше, при использовании могут приходить "плохие данные", с которыми му не будем знать, что делать.


Также по анализу валидационной выборки можно увидеть разброс в количестве документов для разных запросов (от 6 до 118). Это может негативно сказаться на обучении, ведь тогда модель будет уделять внимание наибольшим группам и не уловить зависимости для маленьких. Для минимизации этой проблемы предлагается мониторить качество не только в целом, но и по подвыборкам разного размера.


#Обучение

In [ ]:
group_train = train_df.sort_values(by="qid").groupby("qid").size().tolist()
group_vali = vali_df.sort_values(by="qid").groupby("qid").size().tolist()
group_test = test_df.sort_values(by="qid").groupby("qid").size().tolist()

In [ ]:
x_train = train_df.sort_values(by="qid").drop(['label', 'qid'], axis=1)
y_train = train_df.sort_values(by="qid")['label']
x_vali = vali_df.sort_values(by="qid").drop(['label', 'qid'], axis=1)
y_vali = vali_df.sort_values(by="qid")['label']

In [ ]:
train_data = Dataset(x_train, label=y_train, group=group_train)
vali_data = Dataset(x_vali, label=y_vali, group=group_vali, reference=train_data)

In [ ]:
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'eval_at': [5, 10],
    'learning_rate': 0.05,
    'verbose': 1,
    'early_stopping_rounds': 50
}

model = train(params,
                  train_data,
                  valid_sets=[vali_data],
                  valid_names=['vali'],
                  num_boost_round=500)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.045905 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9233
[LightGBM] [Info] Number of data points in the train set: 9630, number of used features: 40
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[20]	vali's ndcg@5: 0.735979	vali's ndcg@10: 0.780631


In [ ]:
x_vali = vali_df.drop(['label', 'qid'], axis=1)
y_vali = vali_df['label']
preds_vali = model.predict(x_vali)

qid_vali = vali_df['qid']
vali_results_df = pd.DataFrame({
    'qid': qid_vali,
    'true_label': y_vali,
    'pred_score': preds_vali
})

results_list = []

for qid, group in vali_results_df.groupby('qid'):
    y_true = group['true_label'].values
    y_pred = group['pred_score'].values
    num_docs = len(y_true)
    if num_docs > 1:
        try:
            ndcg_5 = ndcg_score([y_true], [y_pred], k=5)
            ndcg_10 = ndcg_score([y_true], [y_pred], k=10)
        except:
            ndcg_5, ndcg_10 = np.nan, np.nan
    else:
        ndcg_5, ndcg_10 = np.nan, np.nan

    max_label = np.max(y_true)
    min_label = np.min(y_true)
    sum_relevant = np.sum(y_true > 0)

    results_list.append({
        'qid': qid,
        'num_docs': num_docs,
        'max_label': max_label,
        'min_label': min_label,
        'sum_relevant': sum_relevant,
        'ndcg@5': ndcg_5,
        'ndcg@10': ndcg_10
    })

query_results_df = pd.DataFrame(results_list)

print("=== ОБЩАЯ СТАТИСТИКА ПО ВАЛИДАЦИИ ===")
print(f"Всего запросов: {len(query_results_df)}")
print(f"Vali NDCG@5: {query_results_df['ndcg@5'].mean():.4f}")
print(f"Vali NDCG@10: {query_results_df['ndcg@10'].mean():.4f}")
print()

print("=== РАСПРЕДЕЛЕНИЕ NDCG ПО ЗАПРОСАМ ===")
print("NDCG@5:")
print(query_results_df['ndcg@5'].describe())
print("\nNDCG@10:")
print(query_results_df['ndcg@10'].describe())
print()

print("=== АНАЛИЗ ПО КАЧЕСТВУ ЗАПРОСОВ ===")
print("\nЗапросы БЕЗ релевантных документов (max_label = 0):")
no_relevant = query_results_df[query_results_df['max_label'] == 0]
print(f"Количество: {len(no_relevant)}")
print(f"Средний NDCG@5: {no_relevant['ndcg@5'].mean():.4f}")
print(f"Средний NDCG@10: {no_relevant['ndcg@10'].mean():.4f}")

print("\nЗапросы С релевантными документами (max_label > 0):")
has_relevant = query_results_df[query_results_df['max_label'] > 0]
print(f"Количество: {len(has_relevant)}")
print(f"Средний NDCG@5: {has_relevant['ndcg@5'].mean():.4f}")
print(f"Средний NDCG@10: {has_relevant['ndcg@10'].mean():.4f}")

print("\nЗапросы с высокорелевантными документами (max_label = 2):")
has_high_relevant = query_results_df[query_results_df['max_label'] == 2]
print(f"Количество: {len(has_high_relevant)}")
print(f"Средний NDCG@5: {has_high_relevant['ndcg@5'].mean():.4f}")
print(f"Средний NDCG@10: {has_high_relevant['ndcg@10'].mean():.4f}")


=== ОБЩАЯ СТАТИСТИКА ПО ВАЛИДАЦИИ ===
Всего запросов: 157
Vali NDCG@5: 0.5113
Vali NDCG@10: 0.5543

=== РАСПРЕДЕЛЕНИЕ NDCG ПО ЗАПРОСАМ ===
NDCG@5:
count    157.000000
mean       0.511322
std        0.379363
min        0.000000
25%        0.000000
50%        0.565603
75%        0.853932
max        1.000000
Name: ndcg@5, dtype: float64

NDCG@10:
count    157.000000
mean       0.554294
std        0.377174
min        0.000000
25%        0.084299
50%        0.630930
75%        0.906025
max        1.000000
Name: ndcg@10, dtype: float64

=== АНАЛИЗ ПО КАЧЕСТВУ ЗАПРОСОВ ===

Запросы БЕЗ релевантных документов (max_label = 0):
Количество: 37
Средний NDCG@5: 0.0000
Средний NDCG@10: 0.0000

Запросы С релевантными документами (max_label > 0):
Количество: 120
Средний NDCG@5: 0.6690
Средний NDCG@10: 0.7252

Запросы с высокорелевантными документами (max_label = 2):
Количество: 68
Средний NDCG@5: 0.6904
Средний NDCG@10: 0.7335


In [ ]:
final_ndcg_5_vali = query_results_df[query_results_df['max_label'] > 0]['ndcg@5'].mean()
final_ndcg_10_vali = query_results_df[query_results_df['max_label'] > 0]['ndcg@10'].mean()

print(f"Финальный NDCG@5 на валидации (по релевантным запросам): {final_ndcg_5_vali:.4f}")
print(f"Финальный NDCG@10 на валидации (по релевантным запросам): {final_ndcg_10_vali:.4f}")

Финальный NDCG@5 на валидации (по релевантным запросам): 0.6690
Финальный NDCG@10 на валидации (по релевантным запросам): 0.7252


Две верхние ячейки были добавлены после рассчета NDCG для тестовой выборке, где он оказался сильно хуже чем для валидации, это натолкнуло меня на мысль, что LightGBM использует другую технику подсчета NDCG, после просмотра исходного кода оказалось, что для запросов без релевантных документов он устанавливает значение в 1, что сильно преувеличивает качество модели, после этого было принято решение считать NDCG вручную и устанавливать 0 для запросов без релевантных документов

#Тест

In [ ]:
importance_gain = model.feature_importance(importance_type='gain')

feature_names = x_train.columns
feat_imp_df = pd.DataFrame({
    'feature': feature_names,
    'split_importance': model.feature_importance(importance_type='split'),
    'gain_importance': model.feature_importance(importance_type='gain')
})

feat_imp_df = feat_imp_df.sort_values('gain_importance', ascending=False)
feat_imp_df.head(10)

,feature,split_importance,gain_importance
38,feature_39,46,2163.790846
39,feature_40,37,1481.712155
22,feature_23,14,374.719665
37,feature_38,13,356.709263
21,feature_22,35,328.115749
28,feature_29,27,314.152738
18,feature_19,40,294.643490
23,feature_24,24,292.087201
0,feature_1,26,235.740412
24,feature_25,24,223.218480


In [ ]:
x_test = test_df.sort_values(by="qid").drop(['label', 'qid'], axis=1)
y_test = test_df.sort_values(by="qid")['label']

In [ ]:
preds = model.predict(x_test)

In [ ]:
qid_test = test_df['qid']
test_results_df = pd.DataFrame({
    'qid': qid_test,
    'true_label': y_test,
    'pred_score': preds
})

results_list = []

for qid, group in test_results_df.groupby('qid'):
    y_true = group['true_label'].values
    y_pred = group['pred_score'].values
    num_docs = len(y_true)
    if num_docs > 1:
        try:
            ndcg_5 = ndcg_score([y_true], [y_pred], k=5)
            ndcg_10 = ndcg_score([y_true], [y_pred], k=10)
        except:
            ndcg_5, ndcg_10 = np.nan, np.nan
    else:
        ndcg_5, ndcg_10 = np.nan, np.nan

    max_label = np.max(y_true)
    min_label = np.min(y_true)
    sum_relevant = np.sum(y_true > 0)

    results_list.append({
        'qid': qid,
        'num_docs': num_docs,
        'max_label': max_label,
        'min_label': min_label,
        'sum_relevant': sum_relevant,
        'ndcg@5': ndcg_5,
        'ndcg@10': ndcg_10
    })

query_results_df = pd.DataFrame(results_list)

print("=== ОБЩАЯ СТАТИСТИКА ПО ТЕСТУ ===")
print(f"Всего запросов: {len(query_results_df)}")
print(f"Test NDCG@5: {query_results_df['ndcg@5'].mean():.4f}")
print(f"Test NDCG@10: {query_results_df['ndcg@10'].mean():.4f}")
print()

print("=== РАСПРЕДЕЛЕНИЕ NDCG ПО ЗАПРОСАМ ===")
print("NDCG@5:")
print(query_results_df['ndcg@5'].describe())
print("\nNDCG@10:")
print(query_results_df['ndcg@10'].describe())
print()

print("=== АНАЛИЗ ПО КАЧЕСТВУ ЗАПРОСОВ ===")
print("\nЗапросы БЕЗ релевантных документов (max_label = 0):")
no_relevant = query_results_df[query_results_df['max_label'] == 0]
print(f"Количество: {len(no_relevant)}")
print(f"Средний NDCG@5: {no_relevant['ndcg@5'].mean():.4f}")
print(f"Средний NDCG@10: {no_relevant['ndcg@10'].mean():.4f}")

print("\nЗапросы С релевантными документами (max_label > 0):")
has_relevant = query_results_df[query_results_df['max_label'] > 0]
print(f"Количество: {len(has_relevant)}")
print(f"Средний NDCG@5: {has_relevant['ndcg@5'].mean():.4f}")
print(f"Средний NDCG@10: {has_relevant['ndcg@10'].mean():.4f}")

print("\nЗапросы с высокорелевантными документами (max_label = 2):")
has_high_relevant = query_results_df[query_results_df['max_label'] == 2]
print(f"Количество: {len(has_high_relevant)}")
print(f"Средний NDCG@5: {has_high_relevant['ndcg@5'].mean():.4f}")
print(f"Средний NDCG@10: {has_high_relevant['ndcg@10'].mean():.4f}")


=== ОБЩАЯ СТАТИСТИКА ПО ТЕСТУ ===
Всего запросов: 156
Test NDCG@5: 0.2765
Test NDCG@10: 0.3428

=== РАСПРЕДЕЛЕНИЕ NDCG ПО ЗАПРОСАМ ===
NDCG@5:
count    156.000000
mean       0.276483
std        0.297256
min        0.000000
25%        0.000000
50%        0.166639
75%        0.500000
max        1.000000
Name: ndcg@5, dtype: float64

NDCG@10:
count    156.000000
mean       0.342797
std        0.318784
min        0.000000
25%        0.000000
50%        0.391413
75%        0.590241
max        1.000000
Name: ndcg@10, dtype: float64

=== АНАЛИЗ ПО КАЧЕСТВУ ЗАПРОСОВ ===

Запросы БЕЗ релевантных документов (max_label = 0):
Количество: 51
Средний NDCG@5: 0.0000
Средний NDCG@10: 0.0000

Запросы С релевантными документами (max_label > 0):
Количество: 105
Средний NDCG@5: 0.4108
Средний NDCG@10: 0.5093

Запросы с высокорелевантными документами (max_label = 2):
Количество: 63
Средний NDCG@5: 0.4276
Средний NDCG@10: 0.5252


Можно увидеть, что треть всех запросов не имеют релевантнвых документов, соответсвтенно их ndcg портит общую статистику исходя из этого факта было рпешено удалить такие документы из оценик качества.

In [ ]:
final_ndcg_5_test = query_results_df[query_results_df['max_label'] > 0]['ndcg@5'].mean()
final_ndcg_10_test = query_results_df[query_results_df['max_label'] > 0]['ndcg@10'].mean()

print(f"Финальный NDCG@5 на тесте (по релевантным запросам): {final_ndcg_5_test:.4f}")
print(f"Финальный NDCG@10 на тесте (по релевантным запросам): {final_ndcg_10_test:.4f}")

Финальный NDCG@5 на тесте (по релевантным запросам): 0.4108
Финальный NDCG@10 на тесте (по релевантным запросам): 0.5093


#Hyperparameter tuning

In [ ]:
def create_datasets_with_params(feature_pre_filter=True):

    train_data = Dataset(x_train, label=y_train, group=group_train,
                        params={'feature_pre_filter': feature_pre_filter})
    vali_data = Dataset(x_vali, label=y_vali, group=group_vali,
                       params={'feature_pre_filter': feature_pre_filter},
                       reference=train_data)
    return train_data, vali_data

def compute_fair_ndcg(y_true, y_pred, qids, k=5):
    ndcg_scores = []

    for qid in np.unique(qids):
        mask = qids == qid
        q_true = y_true[mask]
        q_pred = y_pred[mask]

        if len(q_true) > 1 and np.max(q_true) > 0:
            try:
                ndcg = ndcg_score([q_true], [q_pred], k=k)
                if not np.isnan(ndcg):
                    ndcg_scores.append(ndcg)
            except:
                continue

    return np.mean(ndcg_scores) if ndcg_scores else 0

def objective(trial):
    params = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'eval_at': [5],
        'verbosity': -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 4, 8),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.9),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 1.0, log=True),
        'early_stopping_rounds' : 20
    }

    train_data_optuna, vali_data_optuna = create_datasets_with_params(feature_pre_filter=False)

    model = train(
        params,
        train_data_optuna,
        valid_sets=[vali_data_optuna],
        valid_names=['vali'],
        num_boost_round=500,
    )

    vali_preds = model.predict(x_vali, num_iteration=model.best_iteration)

    ndcg_scores = []
    for qid in vali_df['qid'].unique():
        group_mask = vali_df['qid'] == qid
        y_true = vali_df.loc[group_mask, 'label'].values
        y_pred = vali_preds[group_mask]

        # Только группы с релевантными документами
        if len(y_true) > 1 and np.max(y_true) > 0:
            try:
                ndcg = ndcg_score([y_true], [y_pred], k=5)
                if not np.isnan(ndcg):
                    ndcg_scores.append(ndcg)
            except:
                continue

    return 1 - np.mean(ndcg_scores) if ndcg_scores else 1.0

study = optuna.create_study(direction='minimize')

print("Запуск оптимизации гиперпараметров...")
study.optimize(objective, n_trials=500, show_progress_bar=True)

print("\n=== ЛУЧШИЕ ГИПЕРПАРАМЕТРЫ ===")
print(f"Лучшее значение (1 - NDCG@5): {study.best_value:.4f}")
print(f"Лучший NDCG@5: {1 - study.best_value:.4f}")
print(f"Лучшие параметры: {study.best_params}")

best_params = study.best_params.copy()
best_params.update({
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'eval_at': [5, 10],
    'verbose': 1,
    'early_stopping_rounds' : 50
})

train_data_final, vali_data_final = create_datasets_with_params(feature_pre_filter=False)

final_model = train(
    best_params,
    train_data_final,
    valid_sets=[vali_data_final],
    valid_names=['vali'],
    num_boost_round=500,
)

vali_preds_final = final_model.predict(x_vali)
final_ndcg_5_vali = compute_fair_ndcg(y_vali, vali_preds_final, qid_vali, 5)
final_ndcg_10_vali = compute_fair_ndcg(y_vali, vali_preds_final, qid_vali, 10)

test_preds_final = final_model.predict(x_test)
tuned_ndcg_5_test = compute_fair_ndcg(y_test, test_preds_final, test_df['qid'], 5)
tuned_ndcg_10_test = compute_fair_ndcg(y_test, test_preds_final, test_df['qid'], 10)

print(f"\n=== ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ ===")
print(f"Vali NDCG@5: {final_ndcg_5_vali:.4f}")
print(f"Vali NDCG@10: {final_ndcg_10_vali:.4f}")
print(f"Test NDCG@5: {tuned_ndcg_5_test:.4f}")
print(f"Test NDCG@10: {tuned_ndcg_10_test:.4f}")


print(f"\n=== СРАВНЕНИЕ С BASELINE ===")
print(f"Улучшение Test NDCG@5: {tuned_ndcg_5_test - final_ndcg_5_test:.4f}")
print(f"Улучшение Test NDCG@10: {tuned_ndcg_10_test - final_ndcg_10_test:.4f}")



[I 2025-09-23 18:29:03,732] A new study created in memory with name: no-name-a7c242c5-3593-4cc2-b3dd-40865c6855b0


Запуск оптимизации гиперпараметров...


  0%|          | 0/500 [00:00<?, ?it/s]

[I 2025-09-23 18:29:04,206] Trial 0 finished with value: 0.3339070896966738 and parameters: {'learning_rate': 0.017921088388108963, 'num_leaves': 66, 'max_depth': 4, 'min_child_samples': 26, 'subsample': 0.7115600794495935, 'colsample_bytree': 0.7840624183246755, 'reg_alpha': 0.007429925561652699, 'reg_lambda': 0.7114017780263252}. Best is trial 0 with value: 0.3339070896966738.
[I 2025-09-23 18:29:04,614] Trial 1 finished with value: 0.32076469891431947 and parameters: {'learning_rate': 0.04905792934403772, 'num_leaves': 55, 'max_depth': 8, 'min_child_samples': 97, 'subsample': 0.7267105594830727, 'colsample_bytree': 0.8700373459898084, 'reg_alpha': 0.4818084464694116, 'reg_lambda': 0.01091345937979908}. Best is trial 1 with value: 0.32076469891431947.
[I 2025-09-23 18:29:05,024] Trial 2 finished with value: 0.3339773289009573 and parameters: {'learning_rate': 0.03336171853825274, 'num_leaves': 69, 'max_depth': 7, 'min_child_samples': 35, 'subsample': 0.8085973794283847, 'colsample_by

#Финальный тест

In [ ]:
def train_on_all_data_and_test_on_combined():
    """Train on all training+validation data and test on combined test set"""

    all_train_dfs = []
    all_test_dfs = []

    print("Loading data from all folds...")

    # 1. Собираем все тренировочные и тестовые данные
    for fold_id in range(1, 6):
        fold_path = f"/kaggle/input/letor4/MQ2008/Fold{fold_id}"

        # Загружаем train + vali для обучения
        train_df = load_letor_data(os.path.join(fold_path, "train.txt"))
        vali_df = load_letor_data(os.path.join(fold_path, "vali.txt"))

        # Объединяем train + vali
        combined_train_df = pd.concat([train_df, vali_df], ignore_index=True)
        all_train_dfs.append(combined_train_df)

        # Тестовые данные для объединения
        test_df = load_letor_data(os.path.join(fold_path, "test.txt"))
        all_test_dfs.append(test_df)

    # 2. Объединяем все тренировочные данные
    final_train_df = pd.concat(all_train_dfs, ignore_index=True)

    # 3. Объединяем все тестовые данные в один большой набор
    final_test_df = pd.concat(all_test_dfs, ignore_index=True)
    print(f"Объединенный тестовый набор: {len(final_test_df)} записей")
    print(f"Уникальных запросов в тесте: {final_test_df['qid'].nunique()}")

    # 4. Подготовка данных для обучения
    X_train_all = final_train_df.drop(['label', 'qid'], axis=1)
    y_train_all = final_train_df['label']
    group_train_all = final_train_df.groupby('qid').size().tolist()

    train_data_all = Dataset(X_train_all, label=y_train_all, group=group_train_all)

    # 5. Обучаем финальную модель на ВСЕХ данных
    print("Training final model on all data...")

    params = best_params
    print(params)

    final_model = train(
        params,
        train_data_all,
        num_boost_round=50
    )

    # 6. Тестируем на ОБЪЕДИНЕННОМ тестовом наборе
    print("Testing on combined test set...")
    X_test_combined = final_test_df.drop(['label', 'qid'], axis=1)
    y_test_combined = final_test_df['label']
    qid_test_combined = final_test_df['qid']

    test_preds_combined = final_model.predict(X_test_combined)

    # 7. ДЕТАЛЬНЫЙ АНАЛИЗ на объединенном тестовом наборе
    test_results_df = pd.DataFrame({
        'qid': qid_test_combined,
        'true_label': y_test_combined,
        'pred_score': test_preds_combined
    })

    results_list = []

    for qid, group in test_results_df.groupby('qid'):
        y_true = group['true_label'].values
        y_pred = group['pred_score'].values
        num_docs = len(y_true)
        if num_docs > 1:
            try:
                ndcg_5 = ndcg_score([y_true], [y_pred], k=5)
                ndcg_10 = ndcg_score([y_true], [y_pred], k=10)
            except:
                ndcg_5, ndcg_10 = np.nan, np.nan
        else:
            ndcg_5, ndcg_10 = np.nan, np.nan

        max_label = np.max(y_true)
        min_label = np.min(y_true)
        sum_relevant = np.sum(y_true > 0)

        results_list.append({
            'qid': qid,
            'num_docs': num_docs,
            'max_label': max_label,
            'min_label': min_label,
            'sum_relevant': sum_relevant,
            'ndcg@5': ndcg_5,
            'ndcg@10': ndcg_10
        })

    query_results_df = pd.DataFrame(results_list)

    print("\n" + "="*60)
    print("ДЕТАЛЬНЫЙ АНАЛИЗ НА ОБЪЕДИНЕННОМ ТЕСТОВОМ НАБОРЕ")
    print("="*60)

    print(f"Всего запросов: {len(query_results_df)}")
    print(f"Всего документов: {len(final_test_df)}")
    print()

    print("=== ОБЩАЯ СТАТИСТИКА ===")
    print(f"Test NDCG@5 (все запросы): {query_results_df['ndcg@5'].mean():.4f}")
    print(f"Test NDCG@10 (все запросы): {query_results_df['ndcg@10'].mean():.4f}")
    print()

    print("=== РАСПРЕДЕЛЕНИЕ NDCG ПО ЗАПРОСАМ ===")
    print("NDCG@5:")
    print(query_results_df['ndcg@5'].describe())
    print("\nNDCG@10:")
    print(query_results_df['ndcg@10'].describe())
    print()

    print("=== АНАЛИЗ ПО КАЧЕСТВУ ЗАПРОСОВ ===")
    print("\nЗапросы БЕЗ релевантных документов (max_label = 0):")
    no_relevant = query_results_df[query_results_df['max_label'] == 0]
    print(f"Количество: {len(no_relevant)}")
    print(f"Средний NDCG@5: {no_relevant['ndcg@5'].mean():.4f}")
    print(f"Средний NDCG@10: {no_relevant['ndcg@10'].mean():.4f}")

    print("\nЗапросы С релевантными документами (max_label > 0):")
    has_relevant = query_results_df[query_results_df['max_label'] > 0]
    print(f"Количество: {len(has_relevant)}")
    print(f"Средний NDCG@5: {has_relevant['ndcg@5'].mean():.4f}")
    print(f"Средний NDCG@10: {has_relevant['ndcg@10'].mean():.4f}")

    print("\nЗапросы с высокорелевантными документами (max_label = 2):")
    has_high_relevant = query_results_df[query_results_df['max_label'] == 2]
    print(f"Количество: {len(has_high_relevant)}")
    print(f"Средний NDCG@5: {has_high_relevant['ndcg@5'].mean():.4f}")
    print(f"Средний NDCG@10: {has_high_relevant['ndcg@10'].mean():.4f}")

    # 8. ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ (только релевантные запросы)
    final_ndcg_5_test = query_results_df[query_results_df['max_label'] > 0]['ndcg@5'].mean()
    final_ndcg_10_test = query_results_df[query_results_df['max_label'] > 0]['ndcg@10'].mean()

    print("\n" + "="*60)
    print("ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ")
    print("="*60)
    print(f"Финальный NDCG@5 на тесте (по релевантным запросам): {final_ndcg_5_test:.4f}")
    print(f"Финальный NDCG@10 на тесте (по релевантным запросам): {final_ndcg_10_test:.4f}")


    return final_model, query_results_df

# Запускаем
final_model, combined_test_results = train_on_all_data_and_test_on_combined()

# Сохраняем финальную модель
final_model.save_model('final_ranking_model.txt')
print("Final model saved to 'final_ranking_model.txt'")

# Сохраняем результаты анализа
combined_test_results.to_csv('combined_test_analysis.csv', index=False)
print("Test analysis saved to 'combined_test_analysis.csv'")

Loading data from all folds...
Объединенный тестовый набор: 15211 записей
Уникальных запросов в тесте: 784
Training final model on all data...
{'learning_rate': 0.0153887984134294, 'num_leaves': 95, 'max_depth': 6, 'min_child_samples': 93, 'subsample': 0.8551681431606175, 'colsample_bytree': 0.7169072570515173, 'reg_alpha': 0.019332300791012735, 'reg_lambda': 0.0014702412983056412, 'objective': 'lambdarank', 'metric': 'ndcg', 'eval_at': [5, 10], 'verbose': 1}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.035644 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9383
[LightGBM] [Info] Number of data points in the train set: 60844, number of used features: 40
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N